In [4]:
import tkinter as tk
from tkinter import ttk, messagebox
import requests
from PIL import Image, ImageTk


In [ ]:
API_URL = "http://127.0.0.1:8000"

# Style (по ТЗ)
COLOR_BG = "#FFFFFF"
COLOR_BG2 = "#D2DFFF"
COLOR_ACCENT = "#355CBD"
FONT_TITLE = ("Candara", 20, "bold")
FONT_HEADER = ("Candara", 14, "bold")
FONT_MAIN = ("Candara", 12)

In [14]:
def safe_get_json(url, params=None):
    try:
        r = requests.get(url, params=params, timeout=8)
        r.raise_for_status()
        return r.json()
    except Exception as e:
        raise RuntimeError(str(e))


class App:
    def __init__(self, root):
        self.root = root
        root.title("Система продукции — Компания «Комфорт»")
        root.geometry("1200x720")
        root.configure(bg=COLOR_BG)

        # try icon
        try:
            root.iconbitmap("logo.ico")
        except Exception:
            pass

        # Top bar with logo and title
        self.top = tk.Frame(root, bg=COLOR_BG2, height=110)
        self.top.pack(fill="x")
        self._load_logo_and_title()

        # Main area: left menu + content
        self.main = tk.Frame(root, bg=COLOR_BG)
        self.main.pack(fill="both", expand=True)

        self.left = tk.Frame(self.main, bg=COLOR_BG2, width=260)
        self.left.pack(side="left", fill="y")

        self.content = tk.Frame(self.main, bg=COLOR_BG)
        self.content.pack(side="right", fill="both", expand=True)

        self._create_menu()

        # Start on products
        self.screen_products()

    def _load_logo_and_title(self):
        try:
            img = Image.open("logo.png")
            w, h = img.size
            scale = 90.0 / h
            img = img.resize((int(w * scale), 90), Image.LANCZOS)
            self.logo_img = ImageTk.PhotoImage(img)
            tk.Label(self.top, image=self.logo_img, bg=COLOR_BG2).pack(side="left", padx=12, pady=10)
        except Exception:
            tk.Label(self.top, text="Компания «Комфорт»", font=FONT_TITLE, bg=COLOR_BG2, fg=COLOR_ACCENT).pack(side="left", padx=12, pady=20)

        tk.Label(self.top, text="Управление продукцией", bg=COLOR_BG2, fg=COLOR_ACCENT, font=FONT_HEADER).pack(side="left", padx=8)

    def _create_menu(self):
        buttons = [
            ("Продукция", self.screen_products),
            ("Типы продукции", self.screen_product_types),
            ("Материалы", self.screen_materials),
            ("Цеха", self.screen_workshops),
            ("Процесс", self.screen_process),
            ("Время изготовления", self.screen_time),
            ("Расчёт сырья", self.screen_raw)
        ]
        for txt, cmd in buttons:
            btn = tk.Button(self.left, text=txt, command=cmd, bg=COLOR_ACCENT, fg="white", font=FONT_MAIN, width=24)
            btn.pack(pady=8, padx=10)

    def clear(self):
        for w in self.content.winfo_children():
            w.destroy()

    # -------------------- PRODUCTS screen --------------------
    def screen_products(self):
        self.clear()
        tk.Label(self.content, text="Список продукции", font=FONT_TITLE, bg=COLOR_BG, fg=COLOR_ACCENT).pack(pady=10)

        try:
            data = safe_get_json(f"{API_URL}/products")
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось получить данные: {e}")
            return

        if not data:
            tk.Label(self.content, text="Нет данных", bg=COLOR_BG).pack()
            return

        cols = list(data[0].keys())
        tree = ttk.Treeview(self.content, columns=cols, show="headings", height=18)
        tree.pack(fill="both", expand=True, padx=16, pady=8)

        for c in cols:
            tree.heading(c, text=c)
            tree.column(c, anchor="center", width=140)

        for row in data:
            values = [row.get(c) for c in cols]
            tree.insert("", "end", values=values)

        # Buttons panel
        panel = tk.Frame(self.content, bg=COLOR_BG)
        panel.pack(pady=8)
        tk.Button(panel, text="Добавить", command=lambda: self.open_product_editor(None), bg=COLOR_ACCENT, fg="white", font=FONT_MAIN, width=12).pack(side="left", padx=6)
        tk.Button(panel, text="Редактировать", command=lambda: self._edit_selected(tree), bg=COLOR_ACCENT, fg="white", font=FONT_MAIN, width=12).pack(side="left", padx=6)
        tk.Button(panel, text="Удалить", command=lambda: self._delete_selected(tree), bg=COLOR_ACCENT, fg="white", font=FONT_MAIN, width=12).pack(side="left", padx=6)
        tk.Button(panel, text="Процесс", command=lambda: self.open_process_for_selected(tree), bg=COLOR_ACCENT, fg="white", font=FONT_MAIN, width=12).pack(side="left", padx=6)
        tk.Button(panel, text="Время", command=lambda: self._show_time_selected(tree), bg=COLOR_ACCENT, fg="white", font=FONT_MAIN, width=12).pack(side="left", padx=6)

    def _get_tree_selected_obj(self, tree):
        sel = tree.selection()
        if not sel:
            return None
        item = tree.item(sel[0])["values"]
        # get keys (columns) from first row: request current products
        data = safe_get_json(f"{API_URL}/products")
        cols = list(data[0].keys()) if data else []
        return {cols[i]: item[i] for i in range(len(cols))}

    def _edit_selected(self, tree):
        obj = self._get_tree_selected_obj(tree)
        if not obj:
            messagebox.showwarning("Выбор", "Выберите запись")
            return
        self.open_product_editor(obj)

    def _delete_selected(self, tree):
        obj = self._get_tree_selected_obj(tree)
        if not obj:
            messagebox.showwarning("Выбор", "Выберите запись")
            return
        pid = obj.get("product_id")
        pname = obj.get("product_name") or obj.get("product_name")
        if messagebox.askyesno("Удалить", f"Удалить {pname} (ID {pid})?"):
            try:
                r = requests.delete(f"{API_URL}/products/{pid}")
                if r.status_code == 200:
                    messagebox.showinfo("OK", "Удалено")
                    self.screen_products()
                else:
                    messagebox.showerror("Ошибка", r.text)
            except Exception as e:
                messagebox.showerror("Ошибка", str(e))

    # product editor (add/edit)
    def open_product_editor(self, product):
        # product: None for add, or dict for edit (must contain product_id)
        win = tk.Toplevel(self.root)
        win.title("Добавить / Редактировать продукцию")
        win.geometry("520x520")
        win.configure(bg=COLOR_BG)

        tk.Label(win, text="Продукция", font=FONT_HEADER, bg=COLOR_BG).pack(pady=8)

        # fields
        tk.Label(win, text="Название", bg=COLOR_BG).pack(anchor="w", padx=12)
        e_name = tk.Entry(win, font=FONT_MAIN, width=60); e_name.pack(padx=12, pady=4)

        tk.Label(win, text="Артикул", bg=COLOR_BG).pack(anchor="w", padx=12)
        e_article = tk.Entry(win, font=FONT_MAIN, width=40); e_article.pack(padx=12, pady=4)

        tk.Label(win, text="Минимальная цена", bg=COLOR_BG).pack(anchor="w", padx=12)
        e_price = tk.Entry(win, font=FONT_MAIN, width=20); e_price.pack(padx=12, pady=4)

        # dropdowns: types and materials
        try:
            types = safe_get_json(f"{API_URL}/product-types")
        except Exception:
            types = []
        try:
            materials = safe_get_json(f"{API_URL}/material-types")
        except Exception:
            materials = []

        tk.Label(win, text="Тип продукции", bg=COLOR_BG).pack(anchor="w", padx=12)
        cb_type = ttk.Combobox(win, state="readonly", font=FONT_MAIN, width=40)
        cb_type['values'] = [f"{t['product_type_id']} - {t['type_name']}" for t in types]
        cb_type.pack(padx=12, pady=4)

        tk.Label(win, text="Материал", bg=COLOR_BG).pack(anchor="w", padx=12)
        cb_mat = ttk.Combobox(win, state="readonly", font=FONT_MAIN, width=40)
        cb_mat['values'] = [f"{m['material_type_id']} - {m['material_name']}" for m in materials]
        cb_mat.pack(padx=12, pady=4)

        edit_id = None
        if product:
            # product from /products view likely contains keys: product_id, article_number, product_name, product_type, material_name, min_partner_price
            # or if user passed the full product dict from earlier endpoints, adapt accordingly
            edit_id = product.get("product_id") or product.get("product_id")
            e_name.insert(0, product.get("product_name") or product.get("product_name") or "")
            e_article.insert(0, product.get("article_number") or product.get("article_number") or "")
            e_price.insert(0, str(product.get("min_partner_price") or product.get("min_partner_price") or ""))

            # set type by name if possible
            ptype_name = product.get("product_type") or product.get("product_type")
            for t in types:
                if t.get("type_name") == ptype_name:
                    cb_type.set(f"{t['product_type_id']} - {t['type_name']}")
            # set material
            mname = product.get("material_name") or product.get("material_name")
            for m in materials:
                if m.get("material_name") == mname:
                    cb_mat.set(f"{m['material_type_id']} - {m['material_name']}")

        def save_action():
            name = e_name.get().strip()
            art = e_article.get().strip()
            price_s = e_price.get().strip()
            if not name or not art or not price_s:
                messagebox.showerror("Ошибка", "Заполните обязательные поля")
                return
            try:
                price = float(price_s)
                if price < 0:
                    raise ValueError
            except:
                messagebox.showerror("Ошибка", "Неверная цена")
                return
            if not cb_type.get() or not cb_mat.get():
                messagebox.showerror("Ошибка", "Выберите тип и материал")
                return
            type_id = int(cb_type.get().split(" - ")[0])
            mat_id = int(cb_mat.get().split(" - ")[0])
            payload = {
                "product_type_id": type_id,
                "product_name": name,
                "article_number": art,
                "min_partner_price": price,
                "material_type_id": mat_id
            }
            try:
                if edit_id:
                    r = requests.put(f"{API_URL}/products/{edit_id}", json=payload)
                else:
                    r = requests.post(f"{API_URL}/products", json=payload)
                if r.status_code in (200, 201):
                    messagebox.showinfo("Успех", "Сохранено")
                    win.destroy()
                    self.screen_products()
                else:
                    # show server reply
                    try:
                        messagebox.showerror("Ошибка", r.json())
                    except:
                        messagebox.showerror("Ошибка", r.text)
            except Exception as e:
                messagebox.showerror("Ошибка", str(e))

        tk.Button(win, text="Сохранить", bg=COLOR_ACCENT, fg="white", command=save_action, font=FONT_MAIN).pack(pady=12)

    def _show_time_selected(self, tree):
        obj = self._get_tree_selected_obj(tree)
        if not obj:
            messagebox.showwarning("Выбор", "Выберите продукт")
            return
        pid = obj.get("product_id")
        try:
            res = safe_get_json(f"{API_URL}/manufacture-time/{pid}")
            hours = res.get("manufacture_time_hours", 0)
            messagebox.showinfo("Время изготовления", f"Продукт ID {pid}: {hours} часов")
        except Exception as e:
            messagebox.showerror("Ошибка", str(e))

    def _get_tree_selected_obj(self, tree):
        sel = tree.selection()
        if not sel:
            return None
        vals = tree.item(sel[0])["values"]
        data = safe_get_json(f"{API_URL}/products")
        cols = list(data[0].keys())
        return {cols[i]: vals[i] for i in range(len(cols))}

    # -------------------- PRODUCT TYPES screen (CRUD) --------------------
    def screen_product_types(self):
        self.clear()
        tk.Label(self.content, text="Типы продукции", font=FONT_TITLE, bg=COLOR_BG, fg=COLOR_ACCENT).pack(pady=10)

        try:
            data = safe_get_json(f"{API_URL}/product-types")
        except Exception as e:
            messagebox.showerror("Ошибка", str(e)); return

        cols = list(data[0].keys()) if data else ["product_type_id", "type_name", "coefficient"]
        tree = ttk.Treeview(self.content, columns=cols, show="headings", height=18)
        tree.pack(fill="both", expand=True, padx=16, pady=8)

        for c in cols:
            tree.heading(c, text=c); tree.column(c, anchor="center", width=180)

        for row in data:
            tree.insert("", "end", values=[row.get(k) for k in cols])

        panel = tk.Frame(self.content, bg=COLOR_BG)
        panel.pack(pady=8)
        tk.Button(panel, text="Добавить", command=lambda: self._open_type_editor(None), bg=COLOR_ACCENT, fg="white", font=FONT_MAIN).pack(side="left", padx=6)
        tk.Button(panel, text="Редактировать", command=lambda: self._edit_type_selected(tree), bg=COLOR_ACCENT, fg="white", font=FONT_MAIN).pack(side="left", padx=6)
        tk.Button(panel, text="Удалить", command=lambda: self._delete_type_selected(tree), bg=COLOR_ACCENT, fg="white", font=FONT_MAIN).pack(side="left", padx=6)

    def _open_type_editor(self, obj):
        win = tk.Toplevel(self.root)
        win.title("Тип продукции")
        win.geometry("420x260")
        win.configure(bg=COLOR_BG)
        tk.Label(win, text="Тип продукции", font=FONT_HEADER, bg=COLOR_BG).pack(pady=8)
        tk.Label(win, text="Название", bg=COLOR_BG).pack(anchor="w", padx=12)
        e_name = tk.Entry(win, font=FONT_MAIN, width=40); e_name.pack(padx=12, pady=4)
        tk.Label(win, text="Коэффициент", bg=COLOR_BG).pack(anchor="w", padx=12)
        e_coef = tk.Entry(win, font=FONT_MAIN, width=20); e_coef.pack(padx=12, pady=4)

        edit_id = None
        if obj:
            edit_id = obj.get("product_type_id")
            e_name.insert(0, obj.get("type_name") or "")
            e_coef.insert(0, str(obj.get("coefficient") or ""))

        def save():
            name = e_name.get().strip()
            coef_s = e_coef.get().strip()
            if not name or not coef_s:
                messagebox.showerror("Ошибка", "Заполните поля"); return
            try:
                coef = float(coef_s)
                if coef <= 0: raise ValueError
            except:
                messagebox.showerror("Ошибка", "Коэффициент должен быть положительным числом"); return
            payload = {"type_name": name, "coefficient": coef}
            try:
                if edit_id:
                    r = requests.put(f"{API_URL}/product-types/{edit_id}", json=payload)
                else:
                    r = requests.post(f"{API_URL}/product-types", json=payload)
                if r.status_code in (200,201):
                    messagebox.showinfo("OK", "Сохранено"); win.destroy(); self.screen_product_types()
                else:
                    messagebox.showerror("Ошибка", r.text)
            except Exception as e:
                messagebox.showerror("Ошибка", str(e))

        tk.Button(win, text="Сохранить", bg=COLOR_ACCENT, fg="white", font=FONT_MAIN, command=save).pack(pady=8)

    def _edit_type_selected(self, tree):
        sel = tree.selection()
        if not sel:
            messagebox.showwarning("Выбор", "Выберите строку"); return
        vals = tree.item(sel[0])["values"]
        obj = {"product_type_id": vals[0], "type_name": vals[1], "coefficient": vals[2]}
        self._open_type_editor(obj)

    def _delete_type_selected(self, tree):
        sel = tree.selection()
        if not sel:
            messagebox.showwarning("Выбор", "Выберите строку"); return
        vals = tree.item(sel[0])["values"]
        tid = vals[0]; name = vals[1]
        if not messagebox.askyesno("Удалить", f"Удалить тип {name} (ID {tid})?"): return
        try:
            r = requests.delete(f"{API_URL}/product-types/{tid}")
            if r.status_code == 200:
                messagebox.showinfo("OK", "Удалено"); self.screen_product_types()
            else:
                messagebox.showerror("Ошибка", r.text)
        except Exception as e:
            messagebox.showerror("Ошибка", str(e))

    # -------------------- MATERIALS screen --------------------
    def screen_materials(self):
        self.clear()
        tk.Label(self.content, text="Типы материалов", font=FONT_TITLE, bg=COLOR_BG, fg=COLOR_ACCENT).pack(pady=10)
        try:
            data = safe_get_json(f"{API_URL}/material-types")
        except Exception as e:
            messagebox.showerror("Ошибка", str(e)); return
        cols = list(data[0].keys()) if data else ["material_type_id", "material_name", "loss_percentage"]
        tree = ttk.Treeview(self.content, columns=cols, show="headings", height=18)
        tree.pack(fill="both", expand=True, padx=16, pady=8)
        for c in cols:
            tree.heading(c, text=c); tree.column(c, anchor="center", width=180)
        for row in data:
            tree.insert("", "end", values=[row.get(k) for k in cols])

    # -------------------- WORKSHOPS screen --------------------
    def screen_workshops(self):
        self.clear()
        tk.Label(self.content, text="Цеха", font=FONT_TITLE, bg=COLOR_BG, fg=COLOR_ACCENT).pack(pady=10)
        try:
            data = safe_get_json(f"{API_URL}/workshops")
        except Exception as e:
            messagebox.showerror("Ошибка", str(e)); return
        cols = list(data[0].keys()) if data else ["workshop_id", "workshop_name", "category_name", "staff_count"]
        tree = ttk.Treeview(self.content, columns=cols, show="headings", height=18)
        tree.pack(fill="both", expand=True, padx=16, pady=8)
        for c in cols:
            tree.heading(c, text=c); tree.column(c, anchor="center", width=160)
        for row in data:
            tree.insert("", "end", values=[row.get(k) for k in cols])

    # -------------------- PRODUCTION PROCESS screen --------------------
    def screen_process(self):
        self.clear()
        tk.Label(self.content, text="Производственный процесс", font=FONT_TITLE, bg=COLOR_BG, fg=COLOR_ACCENT).pack(pady=10)

        # Load products and workshops
        try:
            prods = safe_get_json(f"{API_URL}/products")
            workshops = safe_get_json(f"{API_URL}/workshops")
        except Exception as e:
            messagebox.showerror("Ошибка", str(e)); return

        frame = tk.Frame(self.content, bg=COLOR_BG)
        frame.pack(fill="both", expand=True, padx=12, pady=8)

        left = tk.Frame(frame, bg=COLOR_BG)
        left.pack(side="left", fill="y", padx=8, pady=8)

        tk.Label(left, text="Продукты", bg=COLOR_BG).pack(anchor="w")
        lb = tk.Listbox(left, width=40, height=20)
        lb.pack(fill="y", expand=True, pady=4)
        for p in prods:
            lb.insert("end", f"{p.get('product_id')} - {p.get('product_name')}")

        right = tk.Frame(frame, bg=COLOR_BG)
        right.pack(side="right", fill="both", expand=True, padx=8, pady=8)
        tk.Label(right, text="Цеха (поставьте галочки / для удаления — выберите связь внизу)", bg=COLOR_BG).pack(anchor="w")

        workshop_vars = {}
        ws_frame = tk.Frame(right, bg=COLOR_BG)
        ws_frame.pack(fill="both", expand=True, pady=6)
        for w in workshops:
            wid = w.get("workshop_id")
            var = tk.IntVar(value=0)
            cb = tk.Checkbutton(ws_frame, text=f"{w.get('workshop_name')} ({w.get('staff_count')} чел.)", variable=var, bg=COLOR_BG, anchor="w")
            cb.pack(anchor="w")
            workshop_vars[wid] = var

        # bottom: existing links table and actions
        bottom = tk.Frame(self.content, bg=COLOR_BG)
        bottom.pack(fill="both", expand=True, padx=12, pady=6)
        tk.Label(bottom, text="Существующие связи (process_id, product_id, workshop_id, hours):", bg=COLOR_BG).pack(anchor="w")

        cols = ("process_id", "product_id", "workshop_id", "manufacturing_hours", "workshop_name")
        proc_table = ttk.Treeview(bottom, columns=cols, show="headings", height=6)
        for c in cols:
            proc_table.heading(c, text=c); proc_table.column(c, anchor="center", width=140)
        proc_table.pack(fill="both", expand=True, pady=6)

        def load_processes_for(pid):
            proc_table.delete(*proc_table.get_children())
            try:
                rows = safe_get_json(f"{API_URL}/production-process/{pid}")
            except Exception as e:
                messagebox.showerror("Ошибка", str(e)); return
            for r in rows:
                proc_table.insert("", "end", values=[r.get(k) for k in cols])

        def on_product_select(evt):
            sel = lb.curselection()
            if not sel: return
            idx = sel[0]
            pid = prods[idx]['product_id']
            # reset checkboxes
            for k in workshop_vars: workshop_vars[k].set(0)
            # load existing processes and set checkboxes for existing workshops
            try:
                rows = safe_get_json(f"{API_URL}/production-process/{pid}")
            except Exception as e:
                messagebox.showerror("Ошибка", str(e)); return
            for r in rows:
                wid = r.get("workshop_id")
                if wid in workshop_vars:
                    workshop_vars[wid].set(1)
            load_processes_for(pid)

        lb.bind("<<ListboxSelect>>", on_product_select)

        def save_bindings():
            sel = lb.curselection()
            if not sel:
                messagebox.showwarning("Ошибка", "Выберите продукт")
                return
            pid = prods[sel[0]]['product_id']
            selected = [wid for wid, var in workshop_vars.items() if var.get() == 1]
            # We'll attempt to add each selected workshop via POST (manufacturing_hours ask user default)
            hours = simpledialog.askfloat("Время (ч)", "Введите время изготовления в часах (по умолчанию 1.0):", minvalue=0.01)
            if hours is None:
                return
            added = 0
            for wid in selected:
                try:
                    r = requests.post(f"{API_URL}/production-process/{pid}", json={"workshop_id": wid, "manufacturing_hours": float(hours)})
                    if r.status_code in (200,201):
                        added += 1
                except:
                    pass
            messagebox.showinfo("OK", f"Попытка добавления связей выполнена. Добавлено: {added}")
            load_processes_for(pid)

        def delete_selected_process():
            sel = proc_table.selection()
            if not sel:
                messagebox.showwarning("Выбор", "Выберите связь для удаления")
                return
            vals = proc_table.item(sel[0])["values"]
            process_id = vals[0]
            if not messagebox.askyesno("Удалить", f"Удалить связь process_id={process_id}?"):
                return
            # Try to call delete endpoint; if not present, show instruction
            try:
                r = requests.delete(f"{API_URL}/production-process/{process_id}")
                if r.status_code == 200:
                    messagebox.showinfo("OK", "Связь удалена")
                    # refresh current product selection
                    selp = lb.curselection()
                    if selp:
                        pid = prods[selp[0]]['product_id']
                        load_processes_for(pid)
                else:
                    # not implemented or error
                    messagebox.showerror("Ошибка", f"Ответ сервера: {r.status_code} {r.text}")
            except Exception as e:
                messagebox.showerror("Ошибка", "Удаление не поддерживается сервером или ошибка сети.\n" + str(e))

        btns = tk.Frame(self.content, bg=COLOR_BG)
        btns.pack(pady=6)
        tk.Button(btns, text="Сохранить связи (POST)", bg=COLOR_ACCENT, fg="white", command=save_bindings).pack(side="left", padx=6)
        tk.Button(btns, text="Удалить связь", bg=COLOR_ACCENT, fg="white", command=delete_selected_process).pack(side="left", padx=6)

    # -------------------- MANUFACTURE TIME --------------------
    def screen_time(self):
        self.clear()
        tk.Label(self.content, text="Расчёт времени изготовления", font=FONT_TITLE, bg=COLOR_BG, fg=COLOR_ACCENT).pack(pady=10)
        try:
            prods = safe_get_json(f"{API_URL}/products")
        except Exception as e:
            messagebox.showerror("Ошибка", str(e)); return
        cb = ttk.Combobox(self.content, values=[f"{p['product_id']} - {p['product_name']}" for p in prods], font=FONT_MAIN, width=60)
        cb.pack(pady=6)
        def calc_time():
            if not cb.get():
                messagebox.showwarning("Ошибка", "Выберите продукт"); return
            pid = int(cb.get().split(" - ")[0])
            try:
                r = safe_get_json(f"{API_URL}/manufacture-time/{pid}")
                hours = r.get("manufacture_time_hours", 0)
                messagebox.showinfo("Время изготовления", f"Продукт ID {pid}: {hours} часов")
            except Exception as e:
                messagebox.showerror("Ошибка", str(e))
        tk.Button(self.content, text="Рассчитать", bg=COLOR_ACCENT, fg="white", command=calc_time).pack(pady=8)

    # -------------------- RAW CALC --------------------
    def screen_raw(self):
        self.clear()
        tk.Label(self.content, text="Расчёт сырья", font=FONT_TITLE, bg=COLOR_BG, fg=COLOR_ACCENT).pack(pady=10)
        try:
            ptypes = safe_get_json(f"{API_URL}/product-types")
            mtypes = safe_get_json(f"{API_URL}/material-types")
        except Exception as e:
            messagebox.showerror("Ошибка", str(e)); return

        tk.Label(self.content, text="Тип продукции", bg=COLOR_BG).pack(anchor="w", padx=12)
        cbp = ttk.Combobox(self.content, values=[f"{p['product_type_id']} - {p['type_name']}" for p in ptypes], font=FONT_MAIN, width=60)
        cbp.pack(padx=12, pady=4)

        tk.Label(self.content, text="Материал", bg=COLOR_BG).pack(anchor="w", padx=12)
        cbm = ttk.Combobox(self.content, values=[f"{m['material_type_id']} - {m['material_name']}" for m in mtypes], font=FONT_MAIN, width=60)
        cbm.pack(padx=12, pady=4)

        tk.Label(self.content, text="Количество (шт)", bg=COLOR_BG).pack(anchor="w", padx=12)
        e_qty = tk.Entry(self.content, font=FONT_MAIN); e_qty.pack(padx=12, pady=4)

        tk.Label(self.content, text="p1 (float)", bg=COLOR_BG).pack(anchor="w", padx=12)
        e_p1 = tk.Entry(self.content, font=FONT_MAIN); e_p1.pack(padx=12, pady=4)

        tk.Label(self.content, text="p2 (float)", bg=COLOR_BG).pack(anchor="w", padx=12)
        e_p2 = tk.Entry(self.content, font=FONT_MAIN); e_p2.pack(padx=12, pady=4)

        def calc_raw():
            try:
                pid = int(cbp.get().split(" - ")[0])
                mid = int(cbm.get().split(" - ")[0])
                qty = int(e_qty.get())
                p1 = float(e_p1.get())
                p2 = float(e_p2.get())
            except Exception:
                messagebox.showerror("Ошибка", "Проверьте ввод"); return
            try:
                r = requests.get(f"{API_URL}/calc-raw", params={"product_type_id": pid, "material_type_id": mid, "quantity": qty, "p1": p1, "p2": p2})
                rj = r.json()
                if rj.get("result") == -1:
                    messagebox.showerror("Ошибка", "Параметры недопустимы")
                else:
                    messagebox.showinfo("Результат", f"Необходимое количество сырья: {rj.get('result')}")
            except Exception as e:
                messagebox.showerror("Ошибка", str(e))

        tk.Button(self.content, text="Рассчитать", bg=COLOR_ACCENT, fg="white", command=calc_raw).pack(pady=8)

# Run
def main():
    root = tk.Tk()
    App = App(root)
    root.mainloop()

if __name__ == "_main_":
    main()